In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df= pd.read_csv('final_merged.csv')
df.head(2)

,Unnamed: 0,video_id,title_of_video,description_video,category_id_video,tags,duration_video,view_count_video,likes_video,Total_Comments,upload_time,channel_title,Comments_Video,channel_id,description_channel,view_count_channel,Total_Subscriber,Total_Videos,created_time
0,0,-gJ1a4qzO58,Adopted by the SMALLEST FAMILY in Minecraft! (...,Adopted by the SMALLEST FAMILY in Minecraft! (...,20,"['minecraft', 'yes smarty pie himlands', 'Adop...",1139,108078,3539,840,2023-03-10T09:32:59Z,Paglaa Tech,['Favrt COLOUR BTAO and SUBSCRIBE krke Heart l...,UC5v-gTzCpaY257sxx-8P_-Q,1000 subscribers - 31/12/2021\n10000 subscribe...,17097445,147000,184,2016-05-16T16:06:36Z
1,1,gSYT7WSfekk,The BIRTH to DEATH of a Minecraft ELEPHANT! 😱 ...,The BIRTH to DEATH of a Minecraft ELEPHANT! 😱 ...,20,"['minecraft', 'yessmartypie himlands', 'The BI...",967,155964,6147,1163,2023-03-12T07:50:52Z,Paglaa Tech,['Pet btao and Subscribe krke Heart lelo \r IN...,UC5v-gTzCpaY257sxx-8P_-Q,1000 subscribers - 31/12/2021\n10000 subscribe...,17097445,147000,184,2016-05-16T16:06:36Z


In [12]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical variables
le = LabelEncoder()
df['title_encoded'] = le.fit_transform(df['title_of_video'])
df['tags_encoded'] = le.fit_transform(df['tags'])
df['description_encoded'] = le.fit_transform(df['description_channel'])

In [13]:
from sklearn.decomposition import PCA

# Create an instance of the PCA class
#pca = PCA(n_components=2)


# Define features to include in PCA
features = ['title_encoded','tags_encoded','description_encoded']

# Standardize features
df[features] = (df[features] - df[features].mean()) / df[features].std()

# Fit PCA
pca = PCA()
pca.fit(df[features])

# Transform data using PCA components
df_pca = pd.DataFrame(pca.transform(df[features]), columns=[f'component_{i+1}' for i in range(len(features))])

# Create a composite growth score based on PCA components
df['growth_score'] = df_pca.sum(axis=1)

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# Scale numerical features
df[['title_encoded','tags_encoded','description_encoded']] = \
    (df[['title_encoded','tags_encoded','description_encoded']] - \
    df[['title_encoded','tags_encoded','description_encoded']].mean()) / \
    df[['title_encoded','tags_encoded','description_encoded']].std()

# Select features for model
X = df[['title_encoded','tags_encoded','description_encoded']]
y = df['growth_score']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Create and fit the Random Forest Regression Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train, y_train)

# Make predictions on test data
y_pred = rf_model.predict(X_test)

# Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('Mean Squared Error: ', mse)
print('R-squared: ', r2)
print('Mean Absolute Error: ', mae)

Mean Squared Error:  0.03923939193802818
R-squared:  0.9871598956506356
Mean Absolute Error:  0.12700168413123616


In [22]:
import numpy as np
accuracy = 1 - mse/np.var(y_test)

# Print the accuracy
print("Accuracy:",accuracy)


Accuracy: 0.9871598956506356
